In [145]:
## Fixed issue: we should consider only first innings
## Pending issue: 
#  -what is the best way to fix division by 0 problems
#  -tau has too many zeros!!


### prep

In [2]:
import pickle
import pandas as pd
import numpy as np
from scipy.stats import dirichlet, multinomial

/var/folders/lz/7tzrmr9x1xs5pplr6wnzp5jr0000gn/T/ipykernel_21694/2079015914.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
with open('df1.pkl', 'rb') as f:
    data = pickle.load(f)
df = pd.DataFrame(data)
df.head(10)

,bowler,batsman,wicket,over,outcome
0,Mortaza_Bangladesh,Warner_Australia,0,1,0
1,Mortaza_Bangladesh,Warner_Australia,0,1,0
2,Mortaza_Bangladesh,Warner_Australia,0,1,1
3,Mortaza_Bangladesh,Watson_Australia,0,1,0
4,Mortaza_Bangladesh,Watson_Australia,0,1,1
5,Mortaza_Bangladesh,Warner_Australia,0,1,0
6,Shafiul_Bangladesh,Watson_Australia,0,2,0
7,Shafiul_Bangladesh,Watson_Australia,0,2,1
8,Shafiul_Bangladesh,Watson_Australia,0,2,0
9,Shafiul_Bangladesh,Watson_Australia,0,2,2


In [4]:
X = np.load("X_mat.npy")

In [63]:
n_mat = X.sum(axis=-1, keepdims=True)
n_mat[n_mat == 0]=1

In [64]:
n_mat.shape

(374, 20, 10, 1)

In [65]:
# emp_prob = X / (n_mat+1.e-15)
emp_prob = X / (n_mat)

# emp_prob.shape
np.max(emp_prob)

1.0

In [ ]:
emp_prob1  = 

In [69]:
print(emp_prob[emp_prob==0].size)

582857


In [67]:
## Delta theorem multipliers
inv_prob = np.array([(1-p)/(p+1.e-15) for p in emp_prob])
inv_prob = inv_prob/(n_mat+1.e-15)
inv_prob.shape

(374, 20, 10, 8)

In [68]:
inv_prob[inv_prob==0].size

1560

### alpha matrix

In [84]:
alpha=[]
v_inv=[]
for i in range(emp_prob.shape[0]):
    alpha_i=[]
    v_inv_i=[]
    for o in range(19):
        val = emp_prob[i,o+1]/(emp_prob[i,o]+1.e-15)
        # print(val.shape)
        alpha_i.append(val)
        v_i = np.square(val)*(inv_prob[i,o+1]+inv_prob[i,o]) 
        v_inv_i.append([1/(np.sqrt(x)+1.e-15) for x in v_i])
        vtemp= np.array(v_inv_i)
        atemp= np.array(alpha_i)
        # print(atemp.size)
        # print((vtemp[vtemp!=0].size))
        # print(len(alpha_i[alpha_i==0]))
        # if(vtemp[vtemp==0].size!=atemp[atemp==0].size):
            # print(len(v_inv_i[v_inv_i==0]))
            # print(len(alpha_i[alpha_i==0]))
            # print(vtemp.size)
        # v_i_root = np.sqrt(v_i)
        # print(v_i_root.shape)
        # v_inv_i.append(v_i_root)
    alpha.append(alpha_i)
    v_inv.append(v_inv_i)
    
alpha = np.array(alpha)
v_inv = np.array(v_inv)
print(alpha.size)
print(v_inv[v_inv==0].size)

v_inv[alpha==0]=0

print(v_inv[v_inv==0].size)

568480
0
553353


In [86]:
den = np.sum(v_inv, axis=0)
den[den==0]=1
num = np.sum(v_inv*alpha, axis=0)
o_trans = num/den

In [109]:
np.max(alpha)

999999999999999.9

### beta matrix

In [90]:
beta=[]
w_inv=[]
for i in range(emp_prob.shape[0]):
    beta_o=[]
    y_inv_o=[]
    for o in range(20):
        beta_i=[]
        v_inv_i=[]
        for w in range(9):
            val = emp_prob[i,o,w+1]/(emp_prob[i,o,w]+1.e-15)
            beta_i.append(val)
            v_i = np.square(val)*(inv_prob[i,o,w+1]+inv_prob[i,o,w]) 
            v_inv_i.append([1/(np.sqrt(x)+1.e-15) for x in v_i])
            # print(val.shape)
        beta_o.append(beta_i)
        y_inv_o.append(v_inv_i)
    beta.append(beta_o)
    w_inv.append(y_inv_o)
    
beta = np.array(beta)
w_inv = np.array(w_inv)
print(w_inv[w_inv==0].size)
print(beta[beta==0].size)
w_inv[beta==0]=0

print(w_inv[w_inv==0].size)
w_inv.shape 

0
525058
525058


(374, 20, 9, 8)

In [106]:
o_trans.max()

999999999999999.9

In [91]:
den1 = np.sum(w_inv, axis=0)
den1[den1==0]=1
w_trans = np.sum(w_inv*beta, axis=0)/den1
w_trans.shape

(20, 9, 8)

In [92]:
np.min(w_trans)

0.0

### tau calc

In [100]:
multiplier_mat = np.ones((2,20,10,8))
for o in range(1,20):
    multiplier_mat[0,o] = multiplier_mat[0,o-1]*o_trans[o-1]
for w in range(1,10):
    multiplier_mat[1,:,w] = multiplier_mat[1,:,w-1]*w_trans[:,w-1]

tau_mat = multiplier_mat[0]*multiplier_mat[1]
print(tau_mat[tau_mat<=1e-10].size)
for j in range(8):
    if tau_mat[6,0,j]!=0:
        # tau_mat[:,:,j][tau_mat[:,:,j]<(1.e-10)]=0
        tau_mat[:,:,j] = tau_mat[:,:,j]/(tau_mat[6,0,j]) # tau_mat[6,0,:]=1
    else:
        print("here")
        # print(tau_mat[:,:,j])
        tau_mat[:,:,j][tau_mat[:,:,j]<(1.e-10)]=0

        # print(tau_mat[:,:,j])
        tau_mat[:,:,j] = (tau_mat[:,:,j])/(tau_mat[6,0,j]+1.e-15) 
        

1289
here
here


In [114]:
tau_mat[tau_mat>2]=2

In [94]:
# tau_mat[18]

### parameter estimation

In [17]:
# Function to calculate the log posterior
def log_posterior(p, X, a, tau, m):
    log_p = np.sum([(np.sum(X[:, :, j]) + a[j] - 1) * np.log(p[j]) for j in range(len(p))])
    log_p -= np.sum([m[o, w] * np.log(np.sum([tau[o, w, j] * p[j] for j in range(len(p))])+1.e-15) for o in range(X.shape[0]) for w in range(X.shape[1])])
    return log_p

# Metropolis within Gibbs sampler
def metropolis_within_gibbs(X, a, tau, m, num_samples, burn_in):
    num_params = len(a)
    samples = np.zeros((num_samples, num_params))
    current_sample = np.ones(num_params) / num_params  # Initial sample
    for i in range(num_samples + burn_in):
        for j in range(num_params):
            proposal = np.random.dirichlet([np.sum(X[:, :, j]) + a[j] for _ in range(num_params)])
            acceptance_ratio = np.exp(log_posterior(proposal, X, a, tau, m) - log_posterior(current_sample, X, a, tau, m))
            if np.random.rand() < acceptance_ratio:
                current_sample = proposal
        if i >= burn_in:
            samples[i - burn_in] = current_sample
    return samples

In [18]:
def met_sampling(i):
    c = 60
    a = c * np.sum(X/(tau_mat+1.e-15), axis=tuple([0,1,2])) 
    a /= np.sum(X/(tau_mat+1.e-15))
    # tau = np.random.rand(3, 4, 8) 
    m = np.sum(X[i], axis=-1)
    num_samples = 1000
    burn_in = 100
    samples = metropolis_within_gibbs(X[i], a, tau_mat, m, num_samples,burn_in)
    return np.mean(samples, axis=0)

In [ ]:
## Probability matrix calc
prob = []
for i in [1,89]: # range(X.shape[0])
    param_est = met_sampling(i)
    print(param_est)
    vals = tau_mat*param_est
    prob.append(vals/(vals.sum(axis=-1, keepdims=True)+1.e-15))

In [101]:
print(tau_mat)

[[[2.16790608e+000 1.00000000e+000 1.17519932e+000 ... 1.00000000e+015
   1.25292762e-001 4.71570971e-001]
  [3.37728557e+000 9.34066022e-001 1.10788333e+001 ... 0.00000000e+000
   0.00000000e+000 2.18101582e+000]
  [1.35091423e+001 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  ...
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]]

 [[1.95901368e+000 1.00000000e+000 1.18358521e+000 ... 3.33486014e+028
   2.44191370e-001 4.69801905e-001]
  [2.03706895e+000 6.03047460e-001 1.04632871e+000 ... 0.00000000e+000
   2.46345613e+013 5.49780806e+013]
  [1.68539180e+015 5.73540336e+014 9.30069605e+014 ... 0.00000000e+000
   0.00000000e+000 1.49503735e+029]
  ...
  [0.00000000e+000 0.00

In [115]:
#watson is i=1
param_est = met_sampling(1)
print(param_est)
vals = tau_mat*param_est
print(vals.shape)
watson = (vals/(vals.sum(axis=-1, keepdims=True)+1.e-15))


[0.23388017 0.36965765 0.11272513 0.01192111 0.05408686 0.01867322
 0.08492467 0.11413118]
(20, 10, 8)


In [104]:
param_est.sum()

1.0000000000000016

In [20]:
print(vals.sum(axis=-1, keepdims=True)+1.e-15)

[[[1.40165947e+97]
  [5.35838863e+79]
  [1.32344538e+54]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]]

 [[5.93115003e+80]
  [8.40762406e+62]
  [7.68356743e+29]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]]

 [[2.27246526e+64]
  [4.42385011e+46]
  [1.28540221e+07]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]]

 [[1.08561446e+48]
  [5.63384141e+30]
  [1.41042538e-03]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]]

 [[5.29725496e+31]
  [1.22371303e+15]
  [7.56312707e-14]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]
  [1.00000000e-15]]

 [[3.22106727e+15]
  [2.61618630e-01]
  

In [21]:
print(vals)

[[[2.63732780e+087 3.60254825e+059 1.14562357e+097 ... 7.33107226e+013
   1.28909602e+096 1.15198798e+096]
  [4.27509219e+071 2.12925965e+043 4.97347028e+079 ... 0.00000000e+000
   0.00000000e+000 2.80024853e+078]
  [1.32344538e+054 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  ...
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]]

 [[1.76662359e+072 6.03971998e+049 5.18538793e+080 ... 1.79037765e-010
   8.59681764e+078 5.01838931e+079]
  [5.78054475e+061 2.68385300e+033 6.17314974e+062 ... 0.00000000e+000
   2.78956000e+054 1.27075997e+055]
  [2.16245299e+023 3.04847346e+014 7.68356501e+029 ... 0.00000000e+000
   0.00000000e+000 1.89814494e+022]
  ...
  [0.00000000e+000 0.00

In [116]:
watson[10,0,:]

array([0.1022708 , 0.26930379, 0.19669999, 0.        , 0.11192563,
       0.        , 0.08362038, 0.23617941])

In [21]:
prob = np.array(prob)

print(prob[:, 6, 0])

[[2.03603208e-01 7.25401872e-01 1.38743765e-03 0.00000000e+00
  6.90841618e-02 0.00000000e+00 4.52163629e-04 7.11566657e-05]
 [2.11299111e-01 4.17770958e-01 1.21587842e-01 0.00000000e+00
  8.34149433e-02 0.00000000e+00 8.66121831e-02 7.93149615e-02]]


In [ ]:
np.save("prob_mat.npy", prob)

In [ ]:
# Modifications: - weigh recent matches more (use decay ratio)